In [1]:
from ibapi.contract import Contract as IBcontract
from ibapi.order import Order
from ibapi.execution import ExecutionFilter
from threading import Thread
from importlib import reload as rel
import numpy as np, pandas as pd, datetime as dt, time
import queue, tools, collections
import LiveMktData as lmd, OrderPlacement as op, AccountPositions as ap

#### Simulated inputs (from extension)

In [2]:
underlying = 'USO'
msg = 'n=entryL1 d=long t=m p=0 q=1'

#### Historical Market Data

In [4]:
try:
    if app is not None:
        app.disconnect()
        time.sleep(1)
except NameError: app = None 
app = tools.TestApp("127.0.0.1", 7497, 1) # Connection
ibcontract, minTick = tools.make_contract(app, underlying)
hist_mkt_data = tools.get_hist_data(app, ibcontract, durationStr='2 D', barSizeSetting='1 hour')

Getting full contract details from the server... 
IB error id -1 errorcode 2104 string Market data farm connection is OK:usfuture
IB error id -1 errorcode 2107 string HMDS data farm connection is inactive but should be available upon demand.ushmds.us
IB error id -1 errorcode 2107 string HMDS data farm connection is inactive but should be available upon demand.njhmds
IB error id -1 errorcode 2107 string HMDS data farm connection is inactive but should be available upon demand.cashhmds
IB error id -1 errorcode 2107 string HMDS data farm connection is inactive but should be available upon demand.ushmds
Getting historical data from the server... could take up to 8 seconds to complete 
IB error id -1 errorcode 2106 string HMDS data farm connection is OK:ushmds.us


#### Live Market Data

In [4]:
try:
    if app is not None:
        app.disconnect()
        time.sleep(1)
except NameError: app = None 
app = tools.TestApp("127.0.0.1", 7497, 2) # Connection

In [5]:
ibcontract, minTick = tools.make_contract(app, underlying)
tickerid = app.start_getting_IB_market_data(ibcontract)

time.sleep(30)

## What have we got so far?
market_data1 = app.get_IB_market_data(tickerid)

print(market_data1[0])

Getting full contract details from the server... 
IB error id -1 errorcode 2104 string Market data farm connection is OK:usfuture
IB error id -1 errorcode 2106 string HMDS data farm connection is OK:ushmds.us
IB error id -1 errorcode 2107 string HMDS data farm connection is inactive but should be available upon demand.njhmds
IB error id -1 errorcode 2107 string HMDS data farm connection is inactive but should be available upon demand.cashhmds
IB error id -1 errorcode 2107 string HMDS data farm connection is inactive but should be available upon demand.ushmds


AttributeError: 'TestApp' object has no attribute '_my_market_data_dict'

In [ ]:
market_data1_as_df = market_data1.as_pdDataFrame()
print(market_data1_as_df)

time.sleep(30)

## stops the stream and returns all the data we've got so far
market_data2 = app.stop_getting_IB_market_data(tickerid)

## glue the data together
market_data2_as_df = market_data2.as_pdDataFrame()
all_market_data_as_df = pd.concat([market_data1_as_df, market_data2_as_df])

## show some quotes
some_quotes = all_market_data_as_df.resample("1S").last()[["bid_size", "bid_price", "ask_price", "ask_size"]]
print(some_quotes.head(10))

## show some trades
some_trades = all_market_data_as_df.resample("10L").last()[["last_trade_price", "last_trade_size"]]
print(some_trades.head(10))

app.disconnect()

#### Order Placement

In [29]:
app = op.TestApp("127.0.0.1", 7497, 3) # Connection
ibcontract, order1 = tools.parse_message(app, underlying, msg)
orderid1 = app.place_new_IB_order(ibcontract, order1, orderid=None)
print("Placed market order, orderid is %d" % orderid1)
app.disconnect() # Disconnection

Getting full contract details from the server... 
IB error id -1 errorcode 2104 string Market data farm connection is OK:cashfarm
12087792,EUR,CASH,,0.0,,,IDEALPRO,,USD,EUR.USD,EUR.USD,False,,combo:
0,0,0: MKT BUY 1@179769313486231570814527423731704356798070567525844996598917476803157260780028538760589558632766878171540458953514382464234321326889464182768467546703537516986049910576551282076245490090389328944075868508455133942304583236903222948165808559332123348274797826204144723168738177180919299881250404026184124858368.000000 
Getting orderid from IB
Using order id of 1
Placed market order, orderid is 1


#### Account Positions

In [ ]:
app = ap.TestApp("127.0.0.1", 7497, 4)

## lets get positions
positions_list = app.get_current_positions()
print(positions_list)

## get the account name from the position
## normally you would know your account name
accountName = positions_list[0][0] #'DU486001'

## and accounting information
accounting_values = app.get_accounting_values(accountName)
print(accounting_values)

## these values are cached
## if we ask again in more than 5 minutes it will update everything
accounting_updates = app.get_accounting_updates(accountName)
print(accounting_updates)

app.disconnect()